# Setup

In [3]:
from IPython.display import Markdown, display

def print_md(string):
    string = string.replace('\ ', '&nbsp;')
    string = string.replace('\t', '&emsp;')
    string = string.replace('\n', '<br>')
    display(Markdown(string))

In [4]:
conda_env = !echo $CONDA_DEFAULT_ENV
conda_env = conda_env[0]

try:
    if conda_env == 'impact-annotator_env':
        raise Exception("⚠️ <span style='color:red'>Please activate the **impact-annotator_env** conda environment to work with this notebook:</span>\n"
                        "\t\t\ \ <span style='color:blue'>$ source activate impact-annotator_env</span>\n"
                        "\t\t\ \ current environment: " + conda_env)
except Exception as e:
    print_md(str(e))

⚠️ <span style='color:red'>Please activate the **impact-annotator_env** conda environment to work with this notebook:</span><br>&emsp;&emsp;&nbsp;&nbsp;<span style='color:blue'>$ source activate impact-annotator_env</span><br>&emsp;&emsp;&nbsp;&nbsp;current environment: impact-annotator_env

In [18]:
impact = pd.read_csv('../../data/final_IMPACT_mutations_180508.txt', sep = '\t')

In [27]:
impact.isnull().sum()

Hugo_Symbol                     0
Chromosome                      0
Start_Position                  0
End_Position                    0
Consequence                     0
Variant_Type                    0
Reference_Allele                0
Tumor_Seq_Allele2               0
Tumor_Sample_Barcode            0
cDNA_change                    95
HGVSp_Short                  2680
t_depth                         0
t_vaf                           0
t_alt_count                     0
n_depth                         0
n_vaf                           0
n_alt_count                     0
t_ref_plus_count                0
t_ref_neg_count                 0
t_alt_plus_count                0
t_alt_neg_count                 0
confidence_class                0
sample_coverage                 0
mut_key                         0
VAG_VT                          0
VAG_GENE                        0
VAG_cDNA_CHANGE                 0
VAG_PROTEIN_CHANGE              0
VAG_EFFECT                      0
VEP_Consequenc